# Part 1: Introduction to *pyDeltaRCM*

*pyDeltaRCM* is a computationally efficient, free and open source, and easy-to-customize numerical delta model based on the original DeltaRCM model design ([MATLAB deltaRCM model](https://csdms.colorado.edu/wiki/Model:DeltaRCM) by Man Liang; [Liang et al., 2015](https://doi.org/10.5194/esurf-3-67-2015)). *pyDeltaRCM* delivers improved model stability and capabilities, infrastructure to support exploration with minimal boilerplate code, and establishes an approach to extending model capabilities that ensures reproducible and comparable studies. 

<figure>
<img src="https://drive.google.com/uc?export=view&id=1jW2lTExdj0CUvymjnt-hVVT7h8JlPLGK" width="600"/>
</figure>



# Aims of this workshop

In Part 1 of this workshop, we will attempt to:

- Provide an overview of how the *pyDeltaRCM* model is structured
- Walk through the use and configuration of standard model parameters
- Start a model run
- Introduce object-oriented concepts of "subclasses" and "hooks"
- Use these concepts to customize the model to have a sloped basin
- Run our customized model

<br>

### Useful Links and Resources

[Documentation](https://deltarcm.org/pyDeltaRCM/index.html) for the *pyDeltaRCM* model is available online and includes instructions on how to [install the model locally](https://deltarcm.org/pyDeltaRCM/meta/installing.html), [a user guide](https://deltarcm.org/pyDeltaRCM/guides/user_guide.html), [example scripts](https://deltarcm.org/pyDeltaRCM/examples/index.html), and much more.
For fans of the [Basic Model Interface (BMI)](https://bmi.readthedocs.io/en/latest/?badge=latest), the *pyDeltaRCM* model is BMI-compatible, with documentation available [here](https://deltarcm.org/BMI_pyDeltaRCM/).

<br>

More generally, for information on the Python programming language we suggest you explore their [documentation](https://www.python.org/doc/).
To manage local programming environments and package installations we recommend checking out [Anaconda](https://www.anaconda.com/).
For good scientific programming practicies, we encourage you to check out some of the material CSDMS has put out, including the "Level Up!" [webinar](https://csdms.colorado.edu/wiki/Webinars) series.



# Step 0: The DeltaRCM Modeling Framework

The original DeltaRCM model was conceived as a "reduced-complexity" model capable of simulating the growth and evolution of a river delta landscape.
Simulation of the movement of water and sediment is discretized into "parcels" of fluid which move through the domain via a weighted random walk.
As these parcels walk about, they modify the flow fields or topography according to empirical rules, derived from physical relationships known to govern hydrodynamics and sediment transport.
For more details on the random walks, we refer you to the [original DeltaRCM paper](https://doi.org/10.5194/esurf-3-67-2015).

<br>

<figure>
<img src="https://raw.githubusercontent.com/DeltaRCM/pyDeltaRCM/develop/assets/private/liang_2015_fig1fig2.png" width="900"/>
<figcaption> NOTE: Figures are from Liang et al., 2015a, ESurf </figcaption>
</figure>

<br>

<figure>
<img src="https://github.com/DeltaRCM/pyDeltaRCM/blob/gh-pages/pyplots/water_tools/run_water_iteration.hires.png?raw=true" width="900"/>
<figcaption> Figure:  example of random walk through initial domain (top) and after delta development (bottom) </figcaption>
</figure>


# Step 1: Install *pyDeltaRCM*

We will use `pip` to install the package into the Colab environment.

To use *pyDeltaRCM* on your local machine you can do the same `pip` installation. If you wish to join our developer community, we encourage you to download the full repository and install the package from source, developer installation instructions are available [here](https://deltarcm.org/pyDeltaRCM/meta/installing.html#developer-installation).

In [ ]:
%%capture

!pip install pydeltarcm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## Running the model in 5 lines of code


The *pyDeltaRCM* model operations are split into many different "methods" (i.e., functions) that represent a logical step in the model routine. 
We've made an effort to name these methods descriptively, so that the high level functionality of each method is immediately clear, but all of the methods are also individually documented (e.g., [water methods are documented here](https://deltarcm.org/pyDeltaRCM/reference/water_tools/index.html#public-api-methods-attached-to-model)).

---
* `init_domain`
* `init_output_infrastructure`
* `update`
  * `solve_water_and_sediment_timestep`
    * `route_water`
      * `init_water_iteration`
      * `run_water_iteration`
      * `compute_free_surface`
      * `finalize_water_iteration`
    * `route_sediment`  
  * `apply_subsidence`
  * `finalize_timestep`
  * `output_data`
  * `output_checkpoint`
* `finalize`
---



_pyDeltaRCM_ is built with an object-oriented interface. So, in the most basic use case you "run the model" by 1) initializing the model, and 2) updating the model.

Let's start by importing the package, and then creating a `DeltaModel`.

In [ ]:
import pyDeltaRCM

delta = pyDeltaRCM.DeltaModel()

Instantiating the `DeltaModel()` without any arguments will use a set of [default parameters](https://deltarcm.org/pyDeltaRCM/reference/model/yaml_defaults.html) to configure the model run. The default options are a reasonable set for exploring some controls of the model, and would work perfectly well for a simple demonstration here.



> [Find the full `DeltaModel` object documentation here](https://deltarcm.org/pyDeltaRCM/_autosummary/pyDeltaRCM.model.DeltaModel.html#pyDeltaRCM.model.DeltaModel)

<br>

---

<br>

The delta model is run forward with a call to the `update()` method. So, we simply create a for loop, and call the update function, and then wrap everything up with a call to `finalize()` the model:

In [ ]:
# note: this cell will take ~1 minute to run

for _ in range(0, 5):
    delta.update()

delta.finalize()

That’s it! You ran the *pyDeltaRCM* model for five timesteps, with just five lines of code.

There's not a lot that has happened over these five timesteps though.




In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.imshow(delta.bed_elevation)
plt.show()

 While we have given considerable effort to optimize *pyDeltaRCM*, the model is not "real time" fast. So below, we are going to change some settings from the default values for today's tutorial, so that you will be able to see a delta evolve in real time.

## Inspecting the `DeltaModel` object

We saw the bed elevation displayed above, but internally this variable is normally referred to as `eta`. Some other important variables names:

* `eta` : bed elevation (m)
* `depth` : flow depth (m)
* `stage` : water surface elevation (m)
* `uw` : along-streamline flow velocity (m/s)
* `qs` : sediment transport volume (m3/s)

Let's see what a few of these look like:

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(delta.eta)  # change this line to view different variables
plt.colorbar(im, ax=ax, shrink=0.6)
plt.show()

To create a smaller delta model that we can see evolve in real time, we will change a number of configuration parameters. We do this by simply passing a keyword and value pair to the model during instantiation.

Any values that are not specified during the model instantiation will take  default values ([default `pyDeltaRCM` model parameters](https://deltarcm.org/pyDeltaRCM/reference/model/yaml_defaults.html)). The input parameter descriptions can be found in the **Attributes** section of the `DeltaModel` documentation [here](https://deltarcm.org/pyDeltaRCM/_autosummary/pyDeltaRCM.model.DeltaModel.html#pydeltarcm-model-deltamodel).

In [ ]:
# create a smaller delta model
delta = pyDeltaRCM.DeltaModel(
    dx=200,                # grid spacing [50 m]
    Np_water=200,          # input water parcels [2000]
    Np_sed=50,             # input sediment parcels [2000]
    C0_percent=0.2,        # input sediment concentation [0.1 %]
    )

# show the initial condition
fig, ax = plt.subplots()
ax.imshow(delta.eta)
plt.show()


Model parameters can also be defined using a YAML configuration file where the user provides a list of model parameters and their values. For more detailed information on how to write and use YAML files to configure *pyDeltaRCM* model runs, check out [the user guide](https://deltarcm.org/pyDeltaRCM/guides/user_guide.html#configuring-an-input-yaml-file).

In [ ]:
# now loop for 500 timesteps, and plot every 5th step
from IPython.display import clear_output

for i in range(500):
  delta.update()

  if (i % 5) == 0:
    clear_output(wait=True)
    fig, ax = plt.subplots()
    im = ax.imshow(delta.eta)
    plt.colorbar(im, ax=ax, shrink=0.6)
    ax.set_title(i)
    plt.show()

delta.finalize()

#### Reproducible model runs

Compare your model result with your neighbors. Are they the same?

Should they be the same?



In [ ]:
fig, ax = plt.subplots()
ax.imshow(delta.eta)

ax.annotate(
    f'$\\eta={delta.eta[10, 20]:.4f}$',
    xy=(20, 10), xytext=(10, 7.5),
    arrowprops=dict(arrowstyle='-|>',
                    connectionstyle='angle3'),
    bbox=dict(boxstyle='square', fc="w"))

plt.show()

*pyDeltaRCM* supports reproducible runs through an optional `seed` input parameter.

It's important to handle pseudo-random numbers carefully in *pyDeltaRCM* to ensure model runs are reproducible. See the [Developer Guide on Reproducibility](https://deltarcm.org/pyDeltaRCM/guides/developer_guide.html#reproducibility) for more information.

## Doing the science

Consider the case where we are a researcher seeking to explore the effects of a receiving basin that is sloped perpendicular to the channel outlet. 

<br>

This researcher asks: **does this sloped basin cause channels to steer towards the deeper water, where compensation is higher?**

<br>

The `DeltaModel` (and *pyDeltaRCM* more generally) was designed for flexibility and extension by users. Supporting arbitrary and imaginative changes to the model routine *is part of the design*. For example, to examine the effect of a sloped basin... 

<br>

This flexibility is achieved by “subclassing” the `DeltaModel` to create a custom model object, and using “hooks” in the model to achieve the desired modifications.

### Subclassing

Subclassing is a standard concept in object-oriented programming, whereby a *subclass* obtains all the functionality of the parent object, and then adds/modifies existing functionality of the parent, to create a new class of object (i.e., the *subclass*). To subclass the `DeltaModel` we define a new Python object class, which inherits from the model class; below is an example of the basic Python syntax needed to create a subclass.

In [ ]:
class SubclassDeltaModel(pyDeltaRCM.DeltaModel):
    def __init__(self, input_file=None):

        # inherit base DeltaModel methods
        super().__init__(input_file)

mdl = SubclassDeltaModel()
mdl

### Hooks

*Hooks* are methods in the model sequence that do nothing by default, but can be augmented to provide arbitrary desired behavior in the model. Hooks have been integrated throughout the model initialization and update sequences, to allow the users to achieve complex behavior at various stages of the model sequence. 

For example, `hook_solve_water_and_sediment_timestep()` is a hook that occurs immediately before the model `solve_water_and_sediment_timestep()` method. The standard is for a hook to describe the function name that it precedes.

To utilize the hooks, we simply define a method in our subclass with the name corresponding to the hook we want to augment to achieve the desired behavior. For example, to change the behavior our sloped model, we need to modify the inital basin configuration.

[A complete list of model hooks is available in the documentation](https://deltarcm.org/pyDeltaRCM/reference/model/model_hooks.html).

#### A simple example for a modifying hook

Let's first modify the `hook_finalize_timestep()` model hook, which is called once per timestep during `update()`. Remember that hooks are called immediately before the method they are named for, so this hook will be executed immediately before [`finalize_timestep()`](https://deltarcm.org/pyDeltaRCM/reference/iteration_tools/index.html#pyDeltaRCM.iteration_tools.iteration_tools.finalize_timestep), which is the method called to clean up the model state after the morphodynamic steps (i.e., water and sediment routing) of the `update()` routine have been executed.

In [ ]:
class SubclassDeltaModel(pyDeltaRCM.DeltaModel):
    def __init__(self, input_file=None):

        # inherit base DeltaModel methods
        super().__init__(input_file)

        # anything here is called during instantiation
        print('Did the hook get called?')

    def hook_finalize_timestep(self):
      print('Yes, the hook got called.')
        

mdl = SubclassDeltaModel()
mdl

In [ ]:
mdl.update()

### A subclass for a sloped basin

In order to create our subclass model with a sloped basin, we need to think about how to achieve our end goal. The `DeltaModel` methods are organized such that they pertain to individual processes (water routing, sediment routing, etc.). To modify the geometry of the _initial_ basin, we need to access methods that are called during model _initialization_. Before starting to look through the *pyDeltaRCM* codebase itself, we can refer to the documentation to get a list of the available [hooks](https://deltarcm.org/pyDeltaRCM/reference/hook_tools/index.html) to see if there is one which suits our needs.

The careful reader might notice a hook called `hook_after_create_domain()`, which sounds a lot like what we might need to use if we wish to modify the initial basin geometry after the model has been initialized. 

<br>

__But what if we wish to test a number of different cross slopes?__

<br>

To allow subclasses to be as reproducible as the vanilla *pyDeltaRCM*, it is possible to both create custom model parameters, and save custom fields to the model output file. So for our sloped basin subclass, we can define a new YAML parameter that defines the cross slope of the basin. To do this, we will need to use the hook called `hook_import_files()`, to modify the list of model parameters expected to be read from any configuration files. An example of how to use this hook is documented [here](https://deltarcm.org/pyDeltaRCM/examples/custom_yaml.html).

In [ ]:
# subclass for sloped basin
class BasinXSlopeModel(pyDeltaRCM.DeltaModel):
    """A subclass of DeltaModel with sloping basin.

    This subclass simply modifies the basin geometry
    before any computation has occurred.
    """

    def __init__(self, input_file=None, **kwargs):
        """Initialize slope Setup."""
        # inherit base DeltaModel methods
        super().__init__(input_file, **kwargs)
        # calling the above will assign the cross slope to self.basin_xslope

        # I.e., the `super().__init__` above takes care of calling each of the
        # hooks below.

    def hook_after_create_domain(self):
        """Modify the basin to have a cross slope."""
        # note, we will use the attribute basin_xslope in calculations
        eta_line = self.basin_xslope * np.arange(0, self.Width, step=self.dx)
        eta_grid = np.tile(eta_line, (self.L - self.L0, 1))
        eta_grid = eta_grid - ((self.basin_xslope * self.Width)/2)  # center at inlet
        self.eta[self.L0:, :] += eta_grid

    def hook_import_files(self):
        """Define the custom YAML parameters."""
        # note, we have to tell the subclass what parameters to expect
        self.subclass_parameters['basin_xslope'] = {
            'type': ['int', 'float'], 'default': 0
        }


Fantastic! Now that we have defined our sloped basin subclass model, we can intialize it and inspect the topography of the basin. We can also initialize a standard *pyDeltaRCM* `DeltaModel` to compare the initial topography of the regular model to our custom subclass.

In [ ]:
# a sloped delta model
delta = BasinXSlopeModel(
    basin_xslope=0.002,
    dx=200,
    Np_water=200,
    Np_sed=50,
    C0_percent=0.2,
    save_eta_grids=True,
    save_dt=0,
    clobber_netcdf=True)

# a regular pyDeltaRCM.DeltaModel
flat_delta = pyDeltaRCM.DeltaModel(
    dx=200,
    Np_water=200,
    Np_sed=50,
    C0_percent=0.2,
    save_dt=0,
    clobber_netcdf=True,
    seed=int(delta.seed))  # initialize with the same random seed used in the slope delta model above


# make a plot to compare the initial basins
fig, ax = plt.subplots(1, 2, figsize=(8, 3))

i0 = ax[0].imshow(delta.eta)
ax[0].set_title('Sloped Basin')
plt.colorbar(i0, ax=ax[0], shrink=0.3)

i1 = ax[1].imshow(flat_delta.eta)
ax[1].set_title('Standard Basin')
plt.colorbar(i1, ax=ax[1], shrink=0.3)

plt.tight_layout()
plt.show()

__Note__: these two models reference _different_ Python objects! One is our custom subclass, and the other is the *pyDeltaRCM* `DeltaModel` class.

In [ ]:
print(type(delta))
print(type(flat_delta))

Importantly, our subclassed model retains all of the core attributes and functionality of the original `DeltaModel`. Beyond being able to simulate the hydrodynamics and sediment transport necessary to generate a delta, things such as the random seed used for the model run are still known. Extending the model in this way (via subclassing), therefore preserves the reproducibility of the original *pyDeltaRCM* code, while allowing users to customize the model to meet their needs.

In [ ]:
print(delta.seed)
print(flat_delta.seed)

In [ ]:
# run for 500 timesteps, showing every 5th step
for i in range(500):
  delta.update()

  if (i % 5) == 0:
    clear_output(wait=True)
    fig, ax = plt.subplots()
    im = ax.imshow(delta.eta)
    plt.colorbar(im, ax=ax, shrink=0.6)
    ax.set_title(i)
    plt.show()

delta.finalize()

## Analyzing *pyDeltaRCM* model results

*pyDeltaRCM* model outputs are saved to netCDF files. The [netCDF](https://www.unidata.ucar.edu/software/netcdf/) file format is one of several standards for storing scientific data. By saving our model outputs in a standard format, they immediately become interoperable with other scientific software - [QGIS](https://www.qgis.org) for example, can directly read *pyDeltaRCM* output netCDF files.

For more specific details about how the *pyDeltaRCM* output file is formatted, check out the [documentation page](https://deltarcm.org/pyDeltaRCM/info/outputfile.html) on this very subject. In the code block below we will show that the netCDF output file exists and holds data about our model run. The second half of this workshop will focus on analyzing model outputs using the new [DeltaMetrics](https://deltarcm.org/DeltaMetrics/) Python package.

In [ ]:
import os

os.path.isfile('./deltaRCM_Output/pyDeltaRCM_output.nc')

import netCDF4 as nc

ds = nc.Dataset('./deltaRCM_Output/pyDeltaRCM_output.nc')
ds


We can access a list of the variables which are stored in the netCDF

In [ ]:
ds.variables.keys()

If we inspect one of them, we can see the data type (float32), the dimensions of the variable (time, x, y), as well as its units (meters). We also get information about its shape which is organized the same way as the dimensions.

In [ ]:
ds['eta']

Using the netCDF file directly we can plot this variable (the model topography)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(ds['eta'][-1, ...])
ax.set_title('Final Saved Timestep')
plt.show()

We are building a package to make analysis easier and more consistent. This package, `DeltaMetrics` will be the focus of the second half of our tutorial today ([link](https://colab.research.google.com/drive/11kO_fqK6DhBRm0yx4db54iV9OY2-3AYF?usp=sharing)).